In [ ]:
import cv2
from tensorflow import keras
from keras.models import Sequential,load_model
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,AveragePooling2D
from keras.utils import np_utils
from keras import losses
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from IPython.display import clear_output as clear
# from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices()) 

In [ ]:
rgb_frame = []
rgb_label = []
gray_frame = []
gray_label = []
for label in range(14):
    for folder in range(2):
        for png_num in range(18000):
            filename = str(label)+'_'+str(png_num+1)+'.png'
            path = 'D:/chess_train_data/chess_train_picture'+str(folder)+'/'
            frame = cv2.imread(path+filename)
            frame = cv2.resize(frame,(56,56),interpolation=cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            gray_label += [label]
            frame = np.array(frame) / 255.0
            gray_frame += [frame]
            print(path+filename)
            clear()
            
gray_label = np.array(gray_label)

In [ ]:
rgb_frame_t = []
rgb_label_t = []
gray_frame_t = []
gray_label_t = []
for label in range(14):
    for folder in range(2):
        for png_num in range(5400):
            filename = str(label)+'_'+str(png_num+18000+1)+'.png'
            path = 'D:/chess_train_data/chess_train_picture'+str(folder)+'/'
            frame = cv2.imread(path+filename)
            frame = cv2.resize(frame,(56,56),interpolation=cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            gray_label_t += [label]
            frame = np.array(frame) / 255.0
            gray_frame_t += [frame]
            print(path+filename)
            clear()
            
gray_label_t = np.array(gray_label_t)

In [ ]:
'''np.save('D:/chess_train_data/chess_train/rgb_frame2',rgb_frame)
np.save('D:/chess_train_data/chess_train/rgb_label2',rgb_label)'''
np.save('D:/chess_train_data/chess_train/gray_frame_mix_56',gray_frame)
np.save('D:/chess_train_data/chess_train/gray_label_mix_56',gray_label)

In [ ]:
gray_frame = np.load('D:/chess_train_data/chess_train/gray_frame_mix_56.npy')
gray_label = np.load('D:/chess_train_data/chess_train/gray_label_mix_56.npy')

In [ ]:
'''np.save('D:/chess_train_data/chess_train/rgb_frame2',rgb_frame)
np.save('D:/chess_train_data/chess_train/rgb_label2',rgb_label)'''
np.save('D:/chess_train_data/chess_train/gray_frame_t_mix_56',gray_frame_t)
np.save('D:/chess_train_data/chess_train/gray_label_t_mix_56',gray_label_t)

In [ ]:
def randSampe(x,y):
    # 根據樣本的數量，建立 1~ len(x) 的數列 a
    a = np.arange(len(x))
    
    # 將數列打亂，以作為 x (樣本) 跟  y (label) 的亂數次序
    random.shuffle(a)
    
    # 將 x,y 與已經打亂的數列 a 合併成 zx, zy 
    zx = zip(a,x)
    zy = zip(a,y)
    
    # 將 zx,zy 進行排序，此時 zx, zy 內的 x, y 陣列，順序已經被打亂  
    zx = sorted(zx,reverse=False)
    zy = sorted(zy,reverse=False)
    
    #將 zx,zy 重新解開，將已經打亂的 x,y 放在 x_train 及 y_train
    _ ,x_train = zip(*zx)
    _ ,y_train = zip(*zy)
    
    return np.array(x_train),np.array(y_train)

In [ ]:
plt.figure(figsize=(10,10))
check_index = 0
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(gray_frame[i+check_index])
    plt.xlabel(gray_label[i+check_index])
plt.show()

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(64,(3,3),strides=(1,1),input_shape=(56,56,1),padding='same',activation='relu'),
    keras.layers.Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1000,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(14,activation='softmax'),
])

In [ ]:
model = Sequential()
model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(56,56,1),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(14,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),loss = 'categorical_crossentropy',metrics=['accuracy'])

In [ ]:
gray_frame1 = np.reshape(gray_frame,[-1,56,56,1])
gray_label1 = np_utils.to_categorical(gray_label, num_classes=14)

In [ ]:
gen = ImageDataGenretor(featurewise_center=False,
                        rotation_range=40, # 角度值，0~180，影象旋轉
                        width_shift_range=0.2, # 水平平移，相對總寬度的比例
                        height_shift_range=0.2, # 垂直平移，相對總高度的比例
                        shear_range=0.2, # 隨機錯切換角度
                        zoom_range=0.2, # 隨機縮放範圍
                        horizontal_flip=True, # 一半影象水平翻轉
                        fill_mode='nearest' # 填充新建立畫素的方法
                       )

In [ ]:
history = model.fit(gray_frame1,gray_label1, epochs=15
                    ,batch_size=360)

In [ ]:
check_history = model.evaluate(gray_frame1, gray_label1)

In [ ]:
check_history

In [ ]:
plt.figure(figsize=(10,10))
predict_index = 0
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])    
    plt.grid(False)
    a = gray_frame[predict_index+i].reshape([-1,56,56,1])
    m = model.predict(a)
    plt.imshow(gray_frame[predict_index+i])
    plt.xlabel(m.argmax())
plt.show()

In [ ]:
model.save('cnn_darkchess_model.h5')
model.save_weights('cnn_darkchess_weights.h5')

In [ ]:
import pandas as pd 
predictions = model.predict_classes(gray_frame1) 
pd.crosstab(gray_label, predictions, rownames=['實際值'], colnames=['預測值'])

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([-0.05, 1.05])
plt.xlim([0, 5])
plt.legend(loc='lower right')

In [ ]:
import cv2

camera_url = "http://192.168.14.56:8081"

# 创建视频捕获对象
cap = cv2.VideoCapture(camera_url)

if not cap.isOpened():
    print("无法打开摄像头")
    exit()

while True:
    # 读取一帧
    ret, frame = cap.read()
    if not ret:
        print("无法接收帧（可能已到视频流的末尾）。正在退出...")
        break

    frame = cv2.resize(frame, (1920, 1080)) 
    # 显示帧
    cv2.imshow('IP Camera Stream', frame)

    # 按下 'Q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
cv2.destroyAllWindows()
